In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import os
import shutil
import scipy.io
import time
import copy
import numpy as np
from tqdm import tqdm


In [ ]:
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!wget https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat

!mkdir -p flowers/flowers-102
!mv imagelabels.mat flowers/flowers-102/

!tar -xzf 102flowers.tgz -C flowers/flowers-102/

--2025-10-15 22:04:10--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://thor.robots.ox.ac.uk/flowers/102/102flowers.tgz [following]
--2025-10-15 22:04:12--  https://thor.robots.ox.ac.uk/flowers/102/102flowers.tgz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/octet-stream]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  15.8MB/s    in 22s     

2025-10-15 22:04:35 (15.2 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]

--2025-10-15 22:04:35--  https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels

In [ ]:
if os.path.exists('flowers_sorted'):
    shutil.rmtree('flowers_sorted')

labels_path = 'flowers/flowers-102/imagelabels.mat'
source_dir = 'flowers/flowers-102/jpg'
target_dir = 'flowers_sorted'

labels = scipy.io.loadmat(labels_path)['labels'][0]

os.makedirs(target_dir, exist_ok=True)
for i in range(len(labels)):
    label = labels[i]
    class_dir = os.path.join(target_dir, str(label))
    os.makedirs(class_dir, exist_ok=True)
    image_filename = f'image_{i+1:05d}.jpg'
    source_path = os.path.join(source_dir, image_filename)
    target_path = os.path.join(class_dir, image_filename)
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)


data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

full_dataset = ImageFolder(root='flowers_sorted', transform=data_transforms['train'])
full_dataset_val = ImageFolder(root='flowers_sorted', transform=data_transforms['val'])

total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

print(f"\nTotal images: {total_size}")
print(f"Splitting into  Train: {train_size}, Validation: {val_size}, Test: {test_size}")

generator = torch.Generator().manual_seed(42)
train_indices, val_indices, test_indices = random_split(range(total_size), [train_size, val_size, test_size], generator=generator)

train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
val_dataset = torch.utils.data.Subset(full_dataset_val, val_indices)
test_dataset = torch.utils.data.Subset(full_dataset_val, test_indices)

batch_size = 32
num_workers = os.cpu_count()
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
}
dataset_sizes = { 'train': len(train_dataset), 'val': len(val_dataset), 'test': len(test_dataset) }

Organizing files...
File organization complete!

Total images: 8189
Splitting into -> Train: 6551, Validation: 818, Test: 820

DataLoaders created successfully with 2 workers!


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
    def forward(self, x):
        identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=102):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
    def _make_layer(self, block, out_channels, num_blocks, stride=1):
        layers = [block(self.in_channels, out_channels, stride)]
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ResNet(ResidualBlock, layers=[2, 2, 2, 2], num_classes=102).to(device)


ResNet model defined and moved to device: cuda:0


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)
print("\nLoss, Optimizer, and Scheduler are ready.")


Loss, Optimizer, and Scheduler are ready.


In [ ]:
def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=25, patience=5):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = np.inf
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}' + '\n' + '-'*10)
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase.capitalize()} Phase"):
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val':
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    patience_counter = 0
                else:
                    patience_counter += 1

        if phase == 'train': scheduler.step()
        if patience_counter >= patience:
            print(f"\nEarly stopping triggered after {patience} epochs with no improvement.")
            break
        print()

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Loss: {best_val_loss:4f}')
    model.load_state_dict(best_model_wts)
    return model

trained_model = train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=25, patience=5)

Epoch 1/25
----------


Train Phase: 100%|██████████| 205/205 [00:32<00:00,  6.22it/s]


Train Loss: 1.0521 Acc: 0.7002


Val Phase: 100%|██████████| 26/26 [00:04<00:00,  5.97it/s]


Val Loss: 1.0415 Acc: 0.7152

Epoch 2/25
----------


...

Val Loss: 0.8067 Acc: 0.7885

Epoch 16/25
----------


Train Phase: 100%|██████████| 205/205 [00:32<00:00,  6.27it/s]


Train Loss: 0.6378 Acc: 0.8200


Val Phase: 100%|██████████| 26/26 [00:04<00:00,  6.10it/s]

Val Loss: 0.7242 Acc: 0.8093

Early stopping triggered after 5 epochs with no improvement.

Training complete in 9m 50s
Best val Loss: 0.710617
